In [46]:
import re
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize
import pymorphy2  # pip install pymorphy2

In [54]:
morph = pymorphy2.MorphAnalyzer()

### prepare stop words

In [2]:
stopword_ru = stopwords.words('russian')
len(stopword_ru)

151

In [3]:
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    for w in f.readlines():
        stopword_ru.append(w)
        
len(stopword_ru)

169

* ## clean text

In [36]:
text=''' Президент Белоруссии Александр Лукашенко выразил резкое недовольство моральным обликом украинских военных, действующих в зоне конфликта на юго-востоке страны. Соответствующее заявление он сделал на пресс-конференции для российских журналистов, передает БЕЛТА.
«Я вам скажу свою позицию: все боятся, что там [на Украине] будет НАТО. Я недавно [президенту России] Владимиру Владимировичу [Путину] сказал: слушай, мы с тобой скоро Господа будем просить, чтобы там было НАТО, а не отмороженные нацмены с ружьем», — сказал Лукашенко.'''


In [71]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
   
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')

    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())

    return text

clean_text(text)

'президент белоруссии александр лукашенко выразил резкое недовольство моральным обликом украинских военных действующих в зоне конфликта на юговостоке страны соответствующее заявление он сделал на прессконференции для российских журналистов передает белта я вам скажу свою позицию все боятся что там на украине будет нато я недавно президенту россии владимиру владимировичу путину сказал слушай мы с тобой скоро господа будем просить чтобы там было нато а не отмороженные нацмены с ружьем  сказал лукашенко'

In [72]:
clean_text=clean_text(text)

* ## lemmatization  

In [41]:
from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

In [73]:
cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка на стоп-слова
        [4] проверка токена с одного символа
        [5] проверка есть ли данное слово в кэше
        [6] лемматизация слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    
    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if not w in stopword_ru: # [3]
            if len(w)>1: # [4]
                if w in cache: # [5]
                    words_lem.append(cache[w])
                else: # [6]
                    temp_cach = cache[w] = morph.parse(w)[0].normal_form
                    words_lem.append(temp_cach)
    return words_lem

In [74]:
print(lemmatization(clean_text))

['президент', 'белоруссия', 'александр', 'лукашенко', 'выразить', 'резкий', 'недовольство', 'моральный', 'облик', 'украинский', 'военный', 'действующий', 'зона', 'конфликт', 'юговосток', 'страна', 'соответствующий', 'заявление', 'сделать', 'прессконференция', 'российский', 'журналист', 'передавать', 'бёлтый', 'сказать', 'позиция', 'бояться', 'украина', 'нато', 'недавно', 'президент', 'россия', 'владимир', 'владимир', 'путин', 'сказать', 'слушай', 'ты', 'скоро', 'господин', 'быть', 'просить', 'нато', 'отморозить', 'нацмен', 'ружьё', 'сказать', 'лукашенко']


In [75]:
lemmatization(['ехали','в'])

['ехать']

---

In [1]:
import preprocessing_tools as pr

In [2]:
text=''' Президент Белоруссии Александр Лукашенко выразил резкое недовольство моральным обликом украинских военных, действующих в зоне конфликта на юго-востоке страны. Соответствующее заявление он сделал на пресс-конференции для российских журналистов, передает БЕЛТА.
«Я вам скажу свою позицию: все боятся, что там [на Украине] будет НАТО. Я недавно [президенту России] Владимиру Владимировичу [Путину] сказал: слушай, мы с тобой скоро Господа будем просить, чтобы там было НАТО, а не отмороженные нацмены с ружьем», — сказал Лукашенко.'''

In [3]:
clean_text = pr.clean_text(text)
clean_text

'президент белоруссии александр лукашенко выразил резкое недовольство моральным обликом украинских военных действующих в зоне конфликта на юговостоке страны соответствующее заявление он сделал на прессконференции для российских журналистов передает белта я вам скажу свою позицию все боятся что там на украине будет нато я недавно президенту россии владимиру владимировичу путину сказал слушай мы с тобой скоро господа будем просить чтобы там было нато а не отмороженные нацмены с ружьем  сказал лукашенко'

In [6]:
print(pr.lemmatization(clean_text))

['президент', 'белоруссия', 'александр', 'лукашенко', 'выразить', 'резкий', 'недовольство', 'моральный', 'облик', 'украинский', 'военный', 'действующий', 'зона', 'конфликт', 'юговосток', 'страна', 'соответствующий', 'заявление', 'сделать', 'прессконференция', 'российский', 'журналист', 'передавать', 'бёлтый', 'сказать', 'позиция', 'бояться', 'украина', 'нато', 'недавно', 'президент', 'россия', 'владимир', 'владимир', 'путин', 'сказать', 'слушай', 'ты', 'скоро', 'господин', 'быть', 'просить', 'нато', 'отморозить', 'нацмен', 'ружьё', 'сказать', 'лукашенко']
